In [1]:
from AlgoTools import AlgoTools
from Trader import Trader
import pandas as pd
import time
import datetime
import pickle
import numpy as np

# Processing the data
- The data has already been grouped into days and converted to date time objects
- Objective is to shuffle the days, then create a train-test split as I will be training on indivuals days

In [87]:
with open('SP-groupedByDays.pkl', 'rb') as f:
    spDaily = pickle.load(f)

In [88]:
with open('spydf.pickle', 'rb') as f:
    tickData = pickle.load(f)

In [89]:
import random
random.shuffle(spDaily)

In [90]:
trainLen = int(len(spDaily)*0.75)

In [91]:
trainData=spDaily[:trainLen]
testData=spDaily[trainLen:]

In [126]:
tempData=spDaily[10:11]

# Creating the Algorithim

In [127]:
algoTools = AlgoTools(tickData)

In [128]:
def Algo(data,r, a_L, a_S, E_e, stopProf, stopLoss):
    #data = tempData
   # r, a_L, a_S, E_e, stopProf, stopLoss = params
    trader = Trader()
    #for each day in the data
    for d in range(len(data)):
        #for each time stamp in the day
        index = 1
        
        for t in range(data[d].index[2] , data[d].index[len(data[d])-1]):
            index = index + 1
            
            #close out because market is closing                
            ti = tickData.loc[t,"Time2"].hour*100 + tickData.loc[t,"Time2"].minute
            if (1513<ti<1520 or 2330<ti<2359):
                if trader.getLastTradePos() != 0:
                    trader.closeOut(data[d].iloc[index,:])
                continue
            
            


            
            #if the acceleration is greater than or equal to acceleration constant Long
            elif (algoTools.getAccel(t) >= a_L ) and trader.getLastTradePos() == 0:
                trader.goLong(data[d].iloc[index,:],"A_L reached")
            
            #same as previous statment but for shorting
            elif  ( algoTools.getAccel(t) <= a_S) and trader.getLastTradePos() == 0:
                trader.goShort(data[d].iloc[index,:],"A_S reached")
                
            
            #stop profit (in points)
            elif (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() >= stopProf ) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] >= stopProf):
                trader.closeOut(data[d].iloc[index,:],"Stop Prof reached")
                
            
            #stop loss (in points)
            elif trader.getLastTradePos() == 1 and (trader.getLastTradePrice()-data[d].loc[t,"price"] ) >= stopLoss:
                trader.closeOut(data[d].iloc[index,:],"Stop Loss reached L")
                
                
            elif trader.getLastTradePos() == -1 and (data[d].loc[t,"price"]-trader.getLastTradePrice()) >= stopLoss:
                trader.closeOut(data[d].iloc[index,:],"Stop Loss reached S")
                


            
                
            
            
                
            
    return trader

In [129]:
trader = Algo(tempData,0.9,0.5,-0.5,0.01,2,1)

In [130]:
trader.getTradeInfo()

{'Total Return': -114.99999999999773, 'Max Return': 100.0, 'Min Return': -50.0}

In [131]:
trader.getLR()

0    0.675676
dtype: float64

In [125]:
trader.getTradeData()

,Position,TradePrice,TradeTime,Method
0,1.0,1252.6,2008-07-28 03:30:05,A_L reached
1,0.0,1251.6,2008-07-28 04:34:59,Stop Loss reached L
2,-1.0,1252.1,2008-07-28 06:21:30,A_S reached
3,0.0,1253.4,2008-07-28 06:21:40,Stop Loss reached S
4,1.0,1253.9,2008-07-28 06:36:13,A_L reached
...,...,...,...,...
509,0.0,1158.2,2010-10-06 06:36:12,Stop Loss reached S
510,-1.0,1157.6,2010-10-06 07:15:00,A_S reached
511,0.0,1155.6,2010-10-06 07:16:20,Stop Prof reached
512,-1.0,1156.0,2010-10-06 08:30:25,A_S reached
